In [1]:
!pip install selenium# add data to keyword field 
wait_for_page_load('//input[@id="sc.keyword"]')  

driver.find_element_by_xpath('//input[@id="sc.keyword"]').send_keys('Data Scientist')

In [2]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

In [3]:
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"

In [44]:
# Comman function for init driver
def init_driver(url):
    chrome_options = Options()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    
    driver.get(url)
    return driver

In [22]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    except NoSuchElementException:
        print("Element not found")

# Q1: Write a python program to scrape data for “Data Analyst” Job position in “Bangalore” location. You have to scrape the job-title, job-location, company_name, experience_required. You have to scrape first 10 jobs data

In [45]:
# init driver
driver = init_driver('https://www.naukri.com/')

In [24]:
# add data to keyword field 
driver.find_element_by_xpath("//form//input[@name='keyword']").send_keys('Data Analyst')

In [25]:
# add data to location filed
driver.find_element_by_xpath("//form//input[@name='location']").send_keys('Bangalore')

In [26]:
# hit search 
driver.find_element_by_xpath("//div[@class='search-btn']/button").click()

In [27]:
# wait for page load
wait_for_page_load('//article[@class="jobTuple bgWhite br4 mb-8"]')    

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//article[@class="jobTuple bgWhite br4 mb-8"]')[:10]
    
list_for_links = []
final_data = []

# Ittrating over box to find details
for one_box in all_box:
    job_title = one_box.find_element_by_xpath(".//a").text
    company_name = one_box.find_element_by_xpath(".//div[contains(@class,'companyInfo')]/a").text
    job_location = one_box.find_element_by_xpath(".//ul/li[contains(@class,'location')]").text 
    experience_required = one_box.find_element_by_xpath(".//ul/li[contains(@class,'experience')]").text 
    
    link_of_job = one_box.find_element_by_xpath(".//a").get_attribute('href')
    list_for_links.append(link_of_job)
    
    final_data.append({
        'job_title':job_title,
        'company_name':company_name,
        'job_location':job_location,
        'experience_required':experience_required
    })
    
job_data = pd.DataFrame(final_data)
job_data

,job_title,company_name,job_location,experience_required
0,Senior Data Analyst,Gaussian Networks Private Limited,"Gurgaon/Gurugram, Bangalore/Bengaluru",3-5 Yrs
1,Tcs Hiring For Senior Data Analyst (bfsi domain),Tata Consultancy Services Ltd.,"Chennai, Bangalore/Bengaluru",6-11 Yrs
2,Business Data Analyst,Trigent Software,Bangalore/Bengaluru,5-10 Yrs
3,Business Data Analyst,Trigent Software,Bangalore/Bengaluru,5-10 Yrs
4,Business Data Analyst - Google Data Studio & SQL,AVE-Promagne,Bangalore/Bengaluru,3-8 Yrs
5,Senior Data Analyst,Virtusa Consulting Services Pvt Ltd,"Hyderabad/Secunderabad, Pune, Gurgaon/Gurugram...",8-12 Yrs
6,Tcs Hiring For MDM (master data management) Da...,Tata Consultancy Services Ltd.,"Chennai, Bangalore/Bengaluru",6-11 Yrs
7,Senior Data Analyst IDAM Services,GlaxoSmithKline Pharmaceuticals Limited,Bangalore/Bengaluru,4-8 Yrs
8,Data Analyst/Sr.Data Engineer,SYREN TECHNOLOGIES PRIVATE LIMITED,"Hyderabad/Secunderabad, Chennai, Bangalore/Ben...",4-9 Yrs
9,SENIOR DATA ANALYST,McAfee Software (India) Pvt. Ltd,Bangalore/Bengaluru,6-8 Yrs


# 2.Please note that you have to scrape full job description. For that you may have to open each job separately as shown below

In [28]:
# Itterate through each link to find the job description
final_jd_list = []
for one_link in list_for_links:
    driver.get(one_link)
    try:
        # Finding for simple page
        job_description = driver.find_element_by_xpath('//section[@class="job-desc"]').text
        
    except:
        # fiding in diffrent way if got an error 
        job_description = driver.find_element_by_xpath('//div[contains(@itemprop,"description")]').text
    
    job_description = job_description.replace('\n', ' ')
    # appending to list
    final_jd_list.append({'new_page_driver':job_description})
    
desc_df = pd.DataFrame(final_jd_list)
job_data['job_description'] = desc_df
job_data

,job_title,company_name,job_location,experience_required,job_description
0,Senior Data Analyst,Gaussian Networks Private Limited,"Gurgaon/Gurugram, Bangalore/Bengaluru",3-5 Yrs,Job description Gauss Networks is seeking a hi...
1,Tcs Hiring For Senior Data Analyst (bfsi domain),Tata Consultancy Services Ltd.,"Chennai, Bangalore/Bengaluru",6-11 Yrs,Job description Roles and Responsibilities 8+...
2,Business Data Analyst,Trigent Software,Bangalore/Bengaluru,5-10 Yrs,Job description Dear Candidates We have a Imm...
3,Business Data Analyst,Trigent Software,Bangalore/Bengaluru,5-10 Yrs,Job description Dear Candidates We have a Imm...
4,Business Data Analyst - Google Data Studio & SQL,AVE-Promagne,Bangalore/Bengaluru,3-8 Yrs,Job description Your responsibilities will inc...
5,Senior Data Analyst,Virtusa Consulting Services Pvt Ltd,"Hyderabad/Secunderabad, Pune, Gurgaon/Gurugram...",8-12 Yrs,Job Description Roles and Responsibilities Tot...
6,Tcs Hiring For MDM (master data management) Da...,Tata Consultancy Services Ltd.,"Chennai, Bangalore/Bengaluru",6-11 Yrs,Job description Roles and Responsibilities De...
7,Senior Data Analyst IDAM Services,GlaxoSmithKline Pharmaceuticals Limited,Bangalore/Bengaluru,4-8 Yrs,Job description Identity and Access Manageme...
8,Data Analyst/Sr.Data Engineer,SYREN TECHNOLOGIES PRIVATE LIMITED,"Hyderabad/Secunderabad, Chennai, Bangalore/Ben...",4-9 Yrs,Job description Responsibilities: Provide inno...
9,SENIOR DATA ANALYST,McAfee Software (India) Pvt. Ltd,Bangalore/Bengaluru,6-8 Yrs,Job description Role Overview: We are looking ...


# Q3: In this question you have to scrape data using the filters available on the webpage as shown below:


The location filter to be used is “Delhi/NCR”
The salary filter to be used is “3-6” lakhs

In [29]:
# init driver
driver.get('https://www.naukri.com/')

In [30]:
# add data to keyword field 
driver.find_element_by_xpath("//form//input[@name='keyword']").send_keys('Data Analyst')

In [31]:
# add data to location filed
driver.find_element_by_xpath("//form//input[@name='location']").send_keys('Bangalore')

In [32]:
# hit search 
driver.find_element_by_xpath("//div[@class='search-btn']/button").click()

In [33]:
# wait for page load
wait_for_page_load("//input[@id='chk-Delhi / NCR-cityTypeGid-']")    


# Appling filters on the page

# appling location filter
driver.find_element_by_xpath("//span[@title='Delhi / NCR']").click()


# wait for page load
wait_for_page_load("//span[@title='3-6 Lakhs']")    


# appling salary filter
driver.find_element_by_xpath("//span[@title='3-6 Lakhs']").click()


# wait for page load
wait_for_page_load('//article[@class="jobTuple bgWhite br4 mb-8"]') 

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//article[@class="jobTuple bgWhite br4 mb-8"]')[:10]
    
list_for_links = []
final_data = []

# Ittrating over box to find details
for one_box in all_box:
    job_title = one_box.find_element_by_xpath(".//a").text
    company_name = one_box.find_element_by_xpath(".//div[contains(@class,'companyInfo')]/a").text
    job_location = one_box.find_element_by_xpath(".//ul/li[contains(@class,'location')]").text 
    experience_required = one_box.find_element_by_xpath(".//ul/li[contains(@class,'experience')]").text 
    
    link_of_job = one_box.find_element_by_xpath(".//a").get_attribute('href')
    list_for_links.append(link_of_job)
    
    final_data.append({
        'job_title':job_title,
        'company_name':company_name,
        'job_location':job_location,
        'experience_required':experience_required
    })
    
job_data = pd.DataFrame(final_data)
job_data

,job_title,company_name,job_location,experience_required
0,Data Analyst SAS+Alteryx,Podium Systems,"Pune, Chennai, Gurgaon/Gurugram, Bangalore/Ben...",4-9 Yrs
1,Data Analyst with SAS & Python,Podium Systems,"Gurgaon/Gurugram, Chennai, Bangalore",4-7 Yrs
2,Stanza Living Hiring For Data Analyst /MIS,Dtwelve Spaces,"Bangalore/Bengaluru, Delhi / NCR",2-5 Yrs
3,Stanza Living Hiring For Data Analyst /MIS,Dtwelve Spaces,"Bangalore/Bengaluru, Delhi / NCR",2-5 Yrs
4,"EY GDS hiring For Data Analyst(SQL,Python acro...",EYGBS,"Kolkata, Hyderabad/Secunderabad, Pune, Chennai...",4-9 Yrs
5,"Data Analyst (Alteryx) , (Datorama)",Omnicom Media Group,"Hyderabad/Secunderabad, Pune, Gurgaon/Gurugram...",3-8 Yrs
6,Data Analyst/Data Scientist,Skillenable Fintech Private Limited,"Bangalore/Bengaluru, Delhi / NCR, Mumbai (All ...",2-5 Yrs
7,Senior Data Analyst,UrbanClap,"Gurgaon/Gurugram, Bangalore/Bengaluru",2-4 Yrs
8,Data Analyst,TMD - The Modern Dimension,"New Delhi, Bangalore/Bengaluru",3-8 Yrs
9,Tcs Hiring For Data Quality Analyst,Tata Consultancy Services Ltd.,"Kolkata, Hyderabad/Secunderabad, Pune, Chennai...",4-9 Yrs


# Q4: Write a python program to scrape data for first 10 job results for Data scientist Designation in Noida location. You have to scrape company_name, No. of days ago when job was posted, Rating of the company.

In [46]:
email = 'imironman@mailinator.com'
password = 'rBrD33sjd7SamBv'

In [47]:
# init driver
driver.get('https://www.glassdoor.co.in/index.htm')

In [48]:
# pasting email and password
driver.find_element_by_xpath('//form[@name="emailSignInForm"]//input[@id="userEmail"]').send_keys(email)
driver.find_element_by_xpath('//form[@name="emailSignInForm"]//input[@id="userPassword"]').send_keys(password)

# click on sing in 
driver.find_element_by_xpath('//form[@name="emailSignInForm"]//button[@type="submit"]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//form[@name="emailSignInForm"]//input[@id="userEmail"]"}
  (Session info: chrome=93.0.4577.63)


In [49]:
# add data to keyword field 
wait_for_page_load('//input[@id="sc.keyword"]')  

driver.find_element_by_xpath('//input[@id="sc.keyword"]').send_keys('Data Scientist')

In [50]:
# add data to location filed
driver.find_element_by_xpath('//input[@id="sc.location"]').send_keys(Keys.CONTROL + "a")
driver.find_element_by_xpath('//input[@id="sc.location"]').send_keys(Keys.DELETE)
driver.find_element_by_xpath('//input[@id="sc.location"]').send_keys('Noida')

In [51]:
# hit search 
driver.find_element_by_xpath('//button[@type="submit" and contains(@class,"search__SearchStyles__newSearchButton")]').click()

In [52]:
# wait for page load
wait_for_page_load('//article[@id="MainCol"]//li')    

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//article[@id="MainCol"]//li')[:10]
    
final_data = []

# Ittrating over box to find details
for one_box in all_box:
    ago_posted = one_box.find_element_by_xpath('.//div[@data-test="job-age"]').text
    company_name = one_box.find_element_by_xpath('.//a[@data-test="job-link"]').text
    try:
        
        rating = one_box.find_element_by_xpath('.//span[contains(@class,"css-19pjha7")]').text 
    except:
        rating = ''
    
    
    final_data.append({
        'ago_posted':ago_posted,
        'company_name':company_name,
        'rating':rating,
    })
    
job_data = pd.DataFrame(final_data)
job_data

,ago_posted,company_name,rating
0,18d,Data Scientist,
1,30d+,Associate Data Scientist,
2,4d,Data Analytics part time job/internship at Noida,
3,30d+,Data Scientist,
4,30d+,,
5,4d,Management Trainee – Data Science,
6,4d,Analyst – Data Science,
7,6d,Data Scientist,
8,30d+,Data Scientist,
9,6d,Business Analyst - Data Scientist,


# Q5: Write a python program to scrape the salary data for Data Scientist designation in Noida location.

In [53]:
#going to salary page
driver.get('https://www.glassdoor.co.in/Salaries/index.htm')

In [54]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="KeywordSearch"]').send_keys('Data Scientist')

In [55]:
# add data to location filed
driver.find_element_by_xpath('//input[@id="LocationSearch"]').clear()
driver.find_element_by_xpath('//input[@id="LocationSearch"]').send_keys('Noida')

In [56]:
# hit search 
driver.find_element_by_xpath('//button[@id="HeroSearchButton"]').click()

In [57]:
# wait for page load
wait_for_page_load('//div[@id="nodeReplace"]//div[@class="py css-17435dd"]')    

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//div[@id="nodeReplace"]//div[@class="py css-17435dd"]')[:10]
    
final_data = []


# Ittrating over box to find details
for one_box in all_box:
    min_salary = one_box.find_elements_by_xpath('.//p')[0].text
    max_salary = one_box.find_elements_by_xpath('.//p')[1].text
    average_salary = one_box.find_elements_by_xpath('.//h3')[1].text
    company_name = one_box.find_elements_by_xpath('.//h3')[0].text
    rating = one_box.find_element_by_xpath('.//span').text
    
    
    final_data.append({
        'min_salary':min_salary,
        'max_salary':max_salary,
        'average_salary':average_salary,
        'company_name':company_name,
        'rating':rating,

    })
    
job_data = pd.DataFrame(final_data)
job_data

,min_salary,max_salary,average_salary,company_name,rating
0,₹4L,₹13L,"₹6,28,021",Tata Consultancy Services,3.9
1,₹1L,₹28L,"₹9,08,246",IBM,3.9
2,₹6L,₹23L,"₹11,93,390",Accenture,4.1
3,₹5L,₹1Cr,"₹12,49,716",Delhivery,3.7
4,₹4L,₹17L,"₹7,58,335",Ericsson-Worldwide,4
5,₹8L,₹16L,"₹12,80,000",UnitedHealth Group,3.7
6,₹8L,₹20L,"₹13,55,086",Optum,3.7
7,₹10L,₹18L,"₹14,55,430",Optum Global Solutions,
8,₹5L,₹15L,"₹8,86,064",Valiance Solutions,4.2
9,₹6L,₹16L,"₹11,10,000",EXL Service,3.6


# Q6 : Scrape data of first 100 sunglasses listings on flipkart.com. You have to scrape four attributes

Brand
Product Description
Price
Discount %

In [58]:
#going to salary page
driver.get('https://www.flipkart.com/')

In [59]:
# close login popup 
wait_for_page_load('//button[text()="✕"]')
driver.find_element_by_xpath('//button[text()="✕"]').click()

In [60]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@name="q"]').send_keys('sunglasses')

In [61]:
# hit search 
driver.find_element_by_xpath('//button[@type="submit"]').click()

In [62]:
product_count = 0 
final_data = []   
while product_count < 99:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[contains(@data-id,"SGL")]')    

    # finding top all boxes 
    all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"SGL")]')

 
    
    # Ittrating over box to find details
    for one_box in all_box:
        
        brand = one_box.find_element_by_xpath('.//div[@class="_2WkVRV"]').text
        
        try:
            product_description = one_box.find_element_by_xpath('.//div[@class="_2B099V"]/a[@class="IRpwTa"]').text
        except:
            product_description = ''
            
        price = one_box.find_elements_by_xpath('.//div[@class="_25b18c"]/div')[0].text
        discount = one_box.find_elements_by_xpath('.//div[@class="_25b18c"]/div')[2].text



        final_data.append({
            'brand':brand,
            'product_description':product_description,
            'price':price,
            'discount':discount,
        })
        
        product_count += 1
        if product_count > 99:
            break
        
    # Go to next page
    
    driver.find_element_by_xpath('//nav/a[@class="_1LKTO3" ]/span[text() = "Next"]').click()
    
job_data = pd.DataFrame(final_data)
job_data

,brand,product_description,price,discount
0,SHAAH COLLECTIONS,"UV Protection, Polarized, Mirrored Rectangular...",₹198,88% off
1,Elligator,UV Protection Round Sunglasses (54),₹295,88% off
2,kingsunglasses,"Mirrored, UV Protection Wayfarer Sunglasses (F...",₹284,89% off
3,PIRASO,UV Protection Aviator Sunglasses (54),₹237,85% off
4,ROZZETTA CRAFT,"UV Protection, Gradient Rectangular Sunglasses...",₹404,79% off
...,...,...,...,...
95,NuVew,"UV Protection Wayfarer, Rectangular Sunglasses...",₹375,64% off
96,PHENOMENAL,"UV Protection, Mirrored Clubmaster Sunglasses ...",₹319,84% off
97,ROZZETTA CRAFT,"UV Protection, Gradient Rectangular Sunglasses...",₹499,77% off
98,ROYAL SON,"UV Protection, Gradient Butterfly Sunglasses (60)",₹664,66% off


# 7 Scrape 100 reviews data from flipkart.com for iphone11 phone. You have to go the link: https://www.flipkart.com/apple-iphone-11-black-64-gb-includes- earpods-power- adapter/p/itm0f37c2240b217?pid=MOBFKCTSVZAXUHGR&lid=LSTMOBFKC TSVZAXUHGREPBFGI&marketplace.¶
Rating
Review_summary
Full review

In [63]:
url = 'https://www.flipkart.com/apple-iphone-11-black-64-gb-includes-earpods-power-adapter/p/itm0f37c2240b217?pid=MOBFKCTSVZAXUHGR&lid=LSTMOBFKCTSVZAXUHGREPBFGI&marketplace'

In [64]:
#going to salary page
driver.get(url)

In [65]:
# get to the all review page 
driver.find_element_by_xpath('//div[@class="_3UAT2v _16PBlm"]/span').click()

In [66]:
product_count = 0 
final_data = []   
while product_count < 99:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[@class="_27M-vq"]')    

    # finding top all boxes 
    all_box = driver.find_elements_by_xpath('//div[@class="_27M-vq"]') 

    # Ittrating over box to find details
    for one_box in all_box:
        
        rating = one_box.find_element_by_xpath('.//div[contains(@class,"_3LWZlK")]').text
        review_summary = one_box.find_element_by_xpath('.//p[@class="_2-N8zT"]').text
        full_review = one_box.find_element_by_xpath('.//div[@class="t-ZTKy"]').text



        final_data.append({
            'rating':rating,
            'review_summary':review_summary,
            'full_review':full_review,
        })
        
        product_count += 1
        if product_count > 99:
            break
        
    # Go to next page
    sleep(5)
    driver.find_element_by_xpath('//nav/a[@class="_1LKTO3" ]/span[text() = "Next"]').click()
    
job_data = pd.DataFrame(final_data)
job_data

,rating,review_summary,full_review
0,5,Brilliant,The Best Phone for the Money\n\nThe iPhone 11 ...
1,5,Simply awesome,Really satisfied with the Product I received.....
2,5,Best in the market!,Great iPhone very snappy experience as apple k...
3,5,Perfect product!,Amazing phone with great cameras and better ba...
4,5,Fabulous!,This is my first iOS phone. I am very happy wi...
...,...,...,...
95,5,Just wow!,Great iphone.\nI am writing this review after ...
96,5,Super!,Did an upgrade from 6s plus to iphone 11.\nAo ...
97,5,Classy product,Gifted my man on his 30th birthday 🎂 He loves ...
98,5,Must buy!,I rate this product 5* as it has got amazing u...


# Q8: Scrape data for first 100 sneakers you find when you visit flipkart.com and search for “sneakers” in the search field. You have to scrape 4 attributes of each sneaker :
Brand
Product Description
Price
discount %

In [67]:
#going to salary page
driver.get('https://www.flipkart.com/')

In [68]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@name="q"]').send_keys('sneakers')

In [69]:
# hit search 
driver.find_element_by_xpath('//button[@type="submit"]').click()

In [70]:
product_count = 0 
final_data = []   
while product_count < 99:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[contains(@data-id,"SHO")]')    

    # finding top all boxes 
    all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"SHO")]')
    
    # Ittrating over box to find details
    for one_box in all_box:
        
        brand = one_box.find_element_by_xpath('.//div[@class="_2WkVRV"]').text
        
        try:
            product_description = one_box.find_element_by_xpath('.//div[@class="_2B099V"]/a[@class="IRpwTa"]').text
        except:
            product_description = ''
            
        price = one_box.find_elements_by_xpath('.//div[@class="_25b18c"]/div')[0].text
        discount = one_box.find_elements_by_xpath('.//div[@class="_25b18c"]/div')[2].text



        final_data.append({
            'brand':brand,
            'product_description':product_description,
            'price':price,
            'discount':discount,
        })
        
        product_count += 1
        if product_count > 99:
            break
        
    # Go to next page
    
    driver.find_element_by_xpath('//nav/a[@class="_1LKTO3" ]/span[text() = "Next"]').click()
    
job_data = pd.DataFrame(final_data)
job_data

,brand,product_description,price,discount
0,Numenzo,Sneakers For Men,₹449,77% off
1,bluemaker,casual for men (blue 10) Sneakers For Men,₹449,55% off
2,Magnolia,Sneakers For Men,₹356,64% off
3,BRUTON,Combo Pack Of 4 Casual Shoes Loafer Shoes Snea...,₹474,86% off
4,Chevit,Perfect & Affordable Combo Pack of 02 Pairs Sn...,₹599,66% off
...,...,...,...,...
95,Chevit,Perfect & Affordable Combo Pack of 03 Pairs Ca...,₹799,69% off
96,Edoeviv,Luxury Branded Fashionable Men's Casual Walkin...,₹499,37% off
97,SPARX,,₹660,17% off
98,BIRDE,Trendy Casual Shoes Sneakers For Men,₹283,43% off


# Q9: Go to the link - https://www.myntra.com/shoes Set Price filter to “Rs. 6649 to Rs. 13099” , Color filter to “Black”, as shown in the below imageWEB SCRAPING ASSIGNMENT-2 1. And then scrape First 100 shoes data you get. The data should include “Brand” of the shoes , Short Shoe description, price of the shoe

In [71]:
#going to salary page
driver.get('https://www.myntra.com/shoes')

In [ ]:
# appling filter to result

# color filter
driver.find_element_by_xpath('//li[@class="colour-listItem"]//label[contains(@class,"common-customCheckbox") and text() = "Black"]').click()

sleep(5)
# price filter
driver.find_element_by_xpath('//ul[@class="price-list"]//li[2]').click()
# driver.find_elements_by_xpath('//ul[@class="price-list"]//label[contains(@class,"common-customCheckbox") ]')[1].click()

In [72]:
product_count = 0 
final_data = []   
while product_count < 99:
    sleep(5)
    # wait for page load
    wait_for_page_load('//li[@class="product-base"]')    

    # finding top all boxes 
    all_box = driver.find_elements_by_xpath('//li[@class="product-base"]')
    
    # Ittrating over box to find details
    for one_box in all_box:
        
        brand = one_box.find_element_by_xpath('.//h3').text
        product_description = one_box.find_element_by_xpath('.//h4[@class="product-product"]').text
        price = one_box.find_elements_by_xpath('.//div[@class="product-price"]')[0].text



        final_data.append({
            'brand':brand,
            'product_description':product_description,
            'price':price,
        })
        
        product_count += 1
        if product_count > 99:
            break
        
    # Go to next page
    
    driver.find_element_by_xpath('//li[@class="pagination-next"]').click()
    
job_data = pd.DataFrame(final_data)
job_data

,brand,product_description,price
0,Red Tape,Men Sneakers,Rs. 1049Rs. 4699(Rs. 3650 OFF)
1,Newfeel By Decathlon,Men Walking Shoes,Rs. 899
2,Puma,Men Running Enzo 2 Shoes,Rs. 2474Rs. 5499(55% OFF)
3,PUMA Motorsport,Unisex Sf Drift Cat 5 Ultra Sneakers,Rs. 3149Rs. 6999(55% OFF)
4,Provogue,Men Walking Sports Shoes,Rs. 699Rs. 3395(Rs. 2696 OFF)
...,...,...,...
95,MANGO,Women Solid Sneakers,Rs. 1436Rs. 3590(60% OFF)
96,Puma,Sneakers,Rs. 1749Rs. 3499(50% OFF)
97,DressBerry,Women Braided Open Toe Flats,Rs. 764Rs. 1699(55% OFF)
98,Fentacia,Men NA Printed Slip-On Sneakers,Rs. 1274Rs. 4249(70% OFF)


# Q10: Go to webpage https://www.amazon.in/ Enter “Laptop” in the search field and then click the search icon. Then set CPU Type filter to “Intel Core i7” and “Intel Core i9” as shown in the below After setting the filters scrape first 10 laptops data. You have to scrape 3 attributes for each laptop:¶
title
Ratings
Price

In [73]:
#going to salary page
driver.get('https://www.amazon.in/')

In [74]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys('Laptop')

# hit search 
driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

In [75]:
# appling filter to result

# i7 filter
driver.find_element_by_xpath('//li[@aria-label="Intel Core i7"]/span').click()

sleep(5)
# i9 filter
driver.find_element_by_xpath('//li[@aria-label="Intel Core i9"]/span').click()

In [76]:
final_data = []   

# wait for page load
wait_for_page_load('//div[@class="a-section a-spacing-medium"]')    

# finding top 10 boxes 
all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')[:10]

# Ittrating over box to find details
for one_box in all_box:

    title = one_box.find_element_by_xpath('.//h2/a/span').text
    
    try:
        ratting = one_box.find_element_by_xpath('.//i/span').get_attribute('innerHTML')
    except NoSuchElementException:
        ratting = 'NA'
        
        
    try:
        price = one_box.find_element_by_xpath('.//div[contains(@class,"a-spacing-top-small")]//span[@class="a-price"]/span[@class="a-offscreen"]').get_attribute('innerHTML')
    except NoSuchElementException:
        price = 'NA'


    final_data.append({
        'title':title,
        'ratting':ratting,
        'price':price,
    })
        

 
job_data = pd.DataFrame(final_data)
job_data

,title,ratting,price
0,Dell XPS 9370 13.3-inch FHD Display Thin & Lig...,2.8 out of 5 stars,"₹1,14,990"
1,HP Envy 11th Gen Core i7 Processor 13.3-inch (...,4.1 out of 5 stars,"₹1,23,350"
2,Mi Notebook Horizon Edition 14 Intel Core i7-1...,4.2 out of 5 stars,"₹57,990"
3,Lenovo IdeaPad S540 11th Gen Intel Core i7 13....,4.3 out of 5 stars,"₹77,990"
4,"ASUS ROG G703GI-E5148T 17.3"" (43.94 cms) FHD 1...",3.9 out of 5 stars,"₹5,56,524"
5,Fujitsu UH-X 11th Gen Intel Core i7 13.3” (33....,4.1 out of 5 stars,"₹1,07,990"
6,HP Pavilion 13(2021) 11th Gen Intel Core i7 La...,5.0 out of 5 stars,"₹92,990"
7,Lenovo IdeaPad Flex 5 11th Gen Intel Core i7 1...,4.1 out of 5 stars,"₹98,500"
8,"MSI GF75 Thin, Intel i7-10750H, 17.3"" (43.9 cm...",4.0 out of 5 stars,"₹74,990"
9,Lenovo IdeaPad Gaming 3 11th Gen Intel Core i7...,4.5 out of 5 stars,"₹84,990"
